# Novel Writing Progress Tracker

## Overview


This Jupyter notebook helps you track the word count of your novel written in a Microsoft Word document.
It logs daily word counts, calculates metrics, and generates visualizations of your writing progress.


## Import Libraries

In [ ]:

import csv
from datetime import datetime
from docx import Document
import pandas as pd
import matplotlib.pyplot as plt


## Functions

### Read Word Count from Document

In [ ]:

def get_word_count(file_path):
    doc = Document(file_path)
    total_words = sum(len(paragraph.text.split()) for paragraph in doc.paragraphs)
    return total_words


### Log Word Count

In [ ]:

def log_word_count(date, word_count, file_path):
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([date, word_count])


In [ ]:
import csv
from datetime import datetime

def log_metrics(date, total_words, daily_increment, estimated_days, estimated_completion_date, file_path):
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([date, total_words, daily_increment, estimated_days, estimated_completion_date])


# Estimate Completion

In [ ]:
def estimate_completion(df, target_word_count):
    total_words = df['Word Count'].iloc[-1]  # Get the latest cumulative word count
    average_daily_words = df['Word Count'].diff().dropna().mean()  # Calculate average daily word count
    remaining_words = max(target_word_count - total_words, 0)  # Calculate remaining words
    if average_daily_words > 0:
        estimated_days = remaining_words / average_daily_words  # Days to completion
        completion_date = datetime.now().date() + pd.Timedelta(days=estimated_days)
        return estimated_days, completion_date
    else:
        return float('inf'), None  # Infinite days if no progress


### Load Data

In [ ]:

def load_data(file_path):
    return pd.read_csv(file_path, names=['Date', 'Word Count'], parse_dates=['Date'])


In [ ]:
def plot_data(df):
    # Calculate daily increments (difference between consecutive days)
    df['Daily Increment'] = df['Word Count'].diff().fillna(0)  # Fill NA for the first entry with 0

    fig, ax = plt.subplots(figsize=(10, 5))
    
    # Plot total word count
    ax.plot(df['Date'], df['Word Count'], marker='o', label='Total Word Count', color='blue')
    
    # Plot daily increments
    ax.plot(df['Date'], df['Daily Increment'], marker='x', linestyle='--', label='Daily Increment', color='red')
    
    ax.set_title('Word Count Progress')
    ax.set_xlabel('Date')
    ax.set_ylabel('Word Count')
    ax.legend()
    ax.grid(True)
    
    plt.savefig('word_count_progress.png')  # Save the figure as a PNG file
    plt.show()



## Main Execution

In [ ]:
# Get today's date in the format 'YYYY-MM-DD'
today_date = datetime.now().strftime('%Y-%m-%d')
# Define the path to your document
doc_path = f'{ your book name }-nightly_{today_date}.docx'
log_path = 'word_count.csv'
today = datetime.now().date()

# Process the document
word_count = get_word_count(doc_path)
log_word_count(today, word_count, log_path)
df = load_data(log_path)
plot_data(df)

# Set target word count, estimate completion, and output stats
target_word_count = 100000  # Example target, set your target
estimated_days, completion_date = estimate_completion(df, target_word_count)
print(f"Target wordcount: {target_word_count}")
print(f"Current wordcount: {word_count:.0f}")
print(f"Latest contribution: {df['Word Count'].diff().iloc[-1]}")
print(f"Precent to completion: {word_count/target_word_count * 100:.0f}%")
print(f"Estimated days to completion: {estimated_days:.0f}")
print(f"Estimated completion date: {completion_date}")
print(f"Average Daily Words written: {df['Word Count'].diff().dropna().mean()}")
print(f"Highest single-day word count: {df['Word Count'].diff().dropna().max()}")
# Usage
log_metrics(
    datetime.now().date(),
    target_word_count,  # This should be calculated in your script
    df['Word Count'].diff().iloc[-1],   # Calculate this as shown previously
    estimated_days,    # From your completion estimate function
    completion_date,  # From your completion estimate function
    'metrics_log.csv'  # Path to your log file
)

